In [5]:
from langchain_openai import ChatOpenAI
from langchain_core.tools import tool
import requests
from langchain_community.tools import DuckDuckGoSearchRun
from langchain_classic.agents import create_react_agent, AgentExecutor
from langchain_classic import hub
from dotenv import load_dotenv
import os

In [6]:
os.environ['LANGCHAIN_PROJECT'] = "LangChain Agent"

In [7]:
load_dotenv()

True

In [8]:
WEATHERSTACK_API = os.getenv("WEATHERSTACK_API")

In [9]:
search_tool = DuckDuckGoSearchRun()

In [10]:
@tool
def get_weather_data(city: str) -> str:
  """
  This function fetches the current weather data for a given city
  """
  url = f'https://api.weatherstack.com/current?access_key={WEATHERSTACK_API}&query={city}'

  response = requests.get(url)

  return response.json()

In [11]:
llm = ChatOpenAI(
        api_key=os.getenv("OPENAI_API_KEY"),
        base_url="https://openrouter.ai/api/v1"
    )

In [12]:

# Step 2: Pull the ReAct prompt from LangChain Hub
prompt = hub.pull("hwchase17/react")  # pulls the standard ReAct agent prompt


In [13]:
# Step 3: Create the ReAct agent manually with the pulled prompt
agent = create_react_agent(
    llm=llm,
    tools=[search_tool, get_weather_data],
    prompt=prompt
)

In [14]:
# Step 4: Wrap it with AgentExecutor
agent_executor = AgentExecutor(
    agent=agent,
    tools=[search_tool, get_weather_data],
    verbose=True,
    max_iterations=5
)


In [18]:
# What is the release date of Dhadak 2?
# What is the current temp of gurgaon
# Identify the birthplace city of Kalpana Chawla (search) and give its current temperature.

# Step 5: Invoke
# response = agent_executor.invoke({"input": "What is the current temp of gurgaon"})
response = agent_executor.invoke({"input": "Identify the birthplace city of Kalpana Chawla and give its current temperature"})




> Entering new AgentExecutor chain...
I should search for the birthplace city of Kalpana Chawla first and then get the current temperature data
Action: duckduckgo_search
Action Input: "Kalpana Chawla birthplace city"3 weeks ago - Chawla was posthumously awarded ... and extraterrestrial landmarks have been named in her honor. Kalpana Chawla was born on March 17, 1962, in Karnal, Haryana, India , to Banarsi Lal Chawla, the owner of a tire manufacturing plant, and Sanjogta Kharbanda.... August 13, 2025 - कल्पना चावला (भारांग: फाल्गुन 26, 1883 / ग्रेगोरी कैलेण्डर : मार्च 17, 1962 - 1 फरवरी 2003), एक भारतीय अमरीकी ... July 21, 2025 - Kalpana Chawla was born in Karnal , a city in the northern Indian state of Haryana. February 28, 2025 - She remains a fixture on lists of the most inspiring Indian women and is held up as an example for young minds to shoot for the stars. Show Full Article · Born in Karnal, Haryana , she lived up to her name ‘Kalpana’, which means ... March 17, 2025 - They we

In [16]:
print(response)

{'input': 'What is the current temp of gurgaon', 'output': 'The current temperature of Gurgaon is 11°C.'}


In [17]:
print(response['output'])

The current temperature of Gurgaon is 11°C.
